# Introduction

The intent of this notebook is to implement decision tree from scratch, following modular structure for each component and function so that it can be used in other modules and can be customized easily.
<br>You may be thinking why to implement from scratch rather than using ready to use available packages from sklearn.
<br>Learning and in future use the same for customized implementing if required is the main idea.

# Background of decision tree

-	Decision trees are a method of splitting the data based on features to either classify or predicting some value.
-	Each branch in a decision tree divides the data into one or two (or several, if the tree is not binary) groups.
-	Each leaf node is allocated with a single label (class or predicted value). 
-	When predicting using the decision tree, the data is allocated to the appropriate leaf node, and the prediction is the label of that leaf node.

# Implementation
<pre>For implementation we need to answer two question - What type of question to ask and when
i.e. out of all the available features which feature to use for splitting the data and when?</pre>

<pre style="font-family:courier;">There are a family of decision tree algorithms that can be used to answer this question, some are
- ID3 (Iterative Dichotomiser 3)
- C4.5 
- C5.0
- CART (Classification and Regression Trees)
</pre>

<pre>Here we will be using CART algorithm for implementation</pre>

###  Some details of CART 
<pre>- CART algorithm produces only binary trees
- It is a greedy algorithm</pre>
#### how it's work
<pre>
- All tree will start with root node(it will receive all the training set)
- Now each node will ask a true/false question for one of the features(which feature we will see later)
- In response to above question we will split the data into two subsets.
- This subset becomes input to two child node that we add to the tree
- The goal of the split is to unmix data, to create purest possible distribution as we go down the tree. (how to check purity we will see later)
- We will keep dividing the data until there is no further question to ask at which point, we will add a leaf node.

<b>Now coming back to the question of splitting the data, what question to ask and when?</b>
- Before we can think of "what question to ask" we need all available question for which we have to find out how many questions can unmix the label or class
- we can find the number of question to ask at any node using metric <b><i>Gini Impurity</i></b>.
- and which question to ask using concept of <b><i>Information Gain</i></b>.
- we will use <i>Gini Impurity</i> and <i>Information Gain</i> to ask best question at each point.

Note: the best question is one that reduces our uncertainity the most, Gini Impurity caculate how much uncertinity is there in the node and Information gain let us know how much uncertinity is reduced by the that question.

</pre>
##### Gini Impurity
<pre>Gini Impurity is a measurement of the likelihood of an incorrect classification of class labels from the data set.</pre>
##### Information Gain
<pre>
Information gain is the reduction in entropy or surprise by transforming a dataset and is often used in training decision trees. Information gain is calculated by comparing the entropy of the dataset before and after a transformation.
</pre>

<pre>
References:
- https://mlexplained.com/2018/01/05/lightgbm-and-xgboost-explained/
- https://www.youtube.com/watch?v=LDRbO9a6XPU
</pre>

In [22]:
# import utils
from utils.is_numeric import is_numeric
from utils.class_counts import class_counts
from utils.gini import gini
from utils.info_gain import info_gain
from utils.unique_vals import unique_vals

In [2]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [3]:
# Column labels.
# These are used only to print the tree.
header = ["color", "diameter", "label"]

In [9]:
class Question:
    """ A Question is used to partition a dataset 

    This class just records a 'column number' and a 'column value'.
    The 'match' method is used to compare the feature value in an example 
    to the feature value store in the question.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        """ compare the feature value in an example to the feature value in this question """
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))
        

In [26]:
# test
q = Question(0,'Green')
q

Is color == Green?

In [8]:
def partition(rows, question):
    """ Partitions a dataset. For each row in the dataset, 
    check if it matches the question if so, 
    add it to true_rows else to fasle_rows"""
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [30]:
def find_best_split(rows, column_number):
    """ Find the best quesiton to ask by iterating over every feature / value
    and calculating the information gain"""
    best_gain = 0 # track of the best Information gain
    best_quetion = None # best question for the gain
    current_uncertainty = gini(rows, column_number) # column = -1 as or target is in last column
    n_features = len(rows[0]) -1 # number of columns
    for col in range(n_features):
        values = unique_vals(rows,col) # unique values in the columns
        for val in values: # for each unique value in the column
            question = Question(col,val)
            
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            
            # skip the split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            
            # calculate information gain on this split
            gain = info_gain(true_rows, false_rows, current_uncertainty, column_number)
            
            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our toy dataset.
            if gain >= best_gain:
                best_gain,best_quetion = gain, question
    
    return best_gain, best_quetion
            

In [31]:
# test
best_gain, best_question = find_best_split(training_data,column_number=-1)
best_question

Is diameter >= 3?

In [27]:
class Leaf:
    """ A Leaf node classifies data.
    This holds a dictionary of class and number of times it appears 
    in the rows from the training data that reaches this leaf.
    e.g Apple:2"""
    def __init__(self, rows):
        self.predictions = class_counts(rows,column_number=-1)

In [28]:
class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question and the two child"""
    
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [34]:
def build_tree(rows):
    """Build the tree
    recursively build the tree"""
    
    # try partitioning the dataset on each of the unique attribute,
    # calculate the information gain and return the question 
    #that produces the hightes gain.
    gain, question = find_best_split(rows, column_number=-1)
    
    # Base case: no further information gain, 
    # since we cannot ask futher question we'll return a leaf.
    if gain==0:
        return Leaf(rows)
    
    # if came upto this point then there are useful feature to partition
    true_rows, false_rows = partition(rows, question)
    
    # Recursively build the true branch
    true_branch = build_tree(true_rows)
    # Recursively build the false branch
    false_branch = build_tree(false_rows)
    
    # Return a Question node.
    # This records the best feature / value to ask at this point, as well as the branches to follow dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [40]:
def print_tree(node, spacing=""):
    """ Funtion to print the tree structure"""
    # base case if its a leaf node
    if isinstance(node, Leaf):
        print(spacing + "Predict",node.predictions)
        return
    # if not leaf print the question
    print(spacing + str(node.question))
    
    # call the function recursively on the true and false branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [37]:
my_tree = build_tree(training_data)

In [41]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [42]:
def classify(row, node):
    """ Based on the rule of recursion, decide which label to assign for current row"""
    if isinstance(node,Leaf):
        return node.predictions
    # decide whether to move left or right
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    

In [43]:
# test
classify(training_data[0], my_tree)

{'Apple': 1}

In [44]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [45]:
print_leaf(classify(training_data[0], my_tree))

{'Apple': '100%'}

In [46]:
print_leaf(classify(training_data[1], my_tree))

{'Apple': '50%', 'Lemon': '50%'}

In [47]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [48]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}


## Testing with Social media data

In [85]:
# test for social media dataset
import pandas as pd
from sklearn.model_selection import train_test_split
df=pd.read_csv('Social_Network_Ads.csv')
print(df.head())
train_df,test_df = train_test_split(df)

    User ID  Gender  Age  EstimatedSalary  Purchased
0  15624510    Male   19            19000          0
1  15810944    Male   35            20000          0
2  15668575  Female   26            43000          0
3  15603246  Female   27            57000          0
4  15804002    Male   19            76000          0


In [ ]:
header = ["Gender","Age","EstimatedSalary","Purchased"]

In [89]:
tree_fil = build_tree(train_df.drop('User ID',axis=1).values)

In [90]:
print_tree(tree_fil)

Is diameter >= 43?
--> True:
  Is label >= 39000?
  --> True:
    Is label >= 86000?
    --> True:
      Is label >= 134000?
      --> True:
        Is label >= 143000?
        --> True:
          Predict {1: 4}
        --> False:
          Is label >= 141000?
          --> True:
            Predict {0: 1}
          --> False:
            Is label >= 138000?
            --> True:
              Predict {1: 2}
            --> False:
              Is diameter >= 51?
              --> True:
                Predict {0: 1}
              --> False:
                Predict {1: 1}
      --> False:
        Is diameter >= 47?
        --> True:
          Predict {1: 19}
        --> False:
          Is label >= 112000?
          --> True:
            Predict {1: 4}
          --> False:
            Predict {0: 1}
    --> False:
      Is diameter >= 53?
      --> True:
        Is label >= 83000?
        --> True:
          Predict {0: 1}
        --> False:
          Predict {1: 5}
      --> False:
  

In [114]:
di={1:2}
# dir(di)
list(di.keys())[0]

1

In [115]:
for row in test_df.drop('User ID',axis=1).values:
    model_pred = classify(row, tree_fil)
    status=""
    if list(model_pred.keys())[0] == row[-1]:
        status="Correct"
    else:
        status="Incorrect"
    print(model_pred)
    print ("Actual: %s. Predicted: %s, Status: %s" %
           (row[-1], print_leaf(model_pred), status))

{0: 7}
Actual: 0. Predicted: {0: '100%'}, Status: Correct
{0: 1}
Actual: 1. Predicted: {0: '100%'}, Status: Incorrect
{0: 123}
Actual: 0. Predicted: {0: '100%'}, Status: Correct
{0: 123}
Actual: 0. Predicted: {0: '100%'}, Status: Correct
{0: 4}
Actual: 1. Predicted: {0: '100%'}, Status: Incorrect
{0: 123}
Actual: 0. Predicted: {0: '100%'}, Status: Correct
{1: 8}
Actual: 1. Predicted: {1: '100%'}, Status: Correct
{1: 5}
Actual: 1. Predicted: {1: '100%'}, Status: Correct
{1: 4}
Actual: 0. Predicted: {1: '100%'}, Status: Incorrect
{1: 2}
Actual: 0. Predicted: {1: '100%'}, Status: Incorrect
{1: 5}
Actual: 1. Predicted: {1: '100%'}, Status: Correct
{0: 33}
Actual: 0. Predicted: {0: '100%'}, Status: Correct
{0: 123}
Actual: 0. Predicted: {0: '100%'}, Status: Correct
{0: 2}
Actual: 1. Predicted: {0: '100%'}, Status: Incorrect
{1: 5}
Actual: 1. Predicted: {1: '100%'}, Status: Correct
{1: 2}
Actual: 0. Predicted: {1: '100%'}, Status: Incorrect
{1: 15}
Actual: 1. Predicted: {1: '100%'}, Status: 